# Kurvanpassning av brusig data
Kurvanpassningen enligt minsta-kvadratmetoden kan användas för godtyckliga modellfunktioner.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 9]
import scipy.optimize as opt
import numpy as np
from IPython.display import display, Math, Markdown

Denna gång utgår vi från en sinuskurva enligt modellfunktionen

$\displaystyle f(t) = A\cdot\sin\left(2\pi\cdot B\cdot t + C \frac{\pi}{180^o}\right)$

In [ ]:
def sinusfkt(t, A, B, C):
    return A * np.sin(2*np.pi * B * t + C * np.pi/180)

Vi skapar en lista `x` av t-värden och en lista `y_sann` av _sanna_ y-värden för funktionen

$\displaystyle f(t) = 5\cdot\sin\left(2\pi\cdot 0.5\cdot t + 30^o \frac{\pi}{180^o}\right)$

In [ ]:
x = np.arange(-2, 2, 0.01)

origParams = [5, 0.5, 30]

y_sann = sinusfkt(x, *origParams)

En snabb ritning av den sanna kurvan ser ut så här

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y_sann, "k-", label = "original", linewidth=2)

Nu skapar vi en lista med normalfördelade slumptal (med $\overline{y} = 0$ och $\sigma=0.5$) som vi sedan lägger på den sanna kurvan för att skapa en brusig signal `y_brus`.

In [ ]:
brus = np.random.normal(loc=0, scale=0.5, size=x.size)
y_brus = y_sann + brus

fig, ax = plt.subplots()
ax.plot(x, y_brus, "b.", label = "med brus", markersize = 4)
ax.plot(x, y_sann, "k-", label = "original", linewidth = 1)

Inför kurvanpassningen så definierar vi en lista med startvärden för `A`, `B` och `C`

In [ ]:
guessParams = [1, 1, 1] # kolla även [1, 0.2, 0]

Och så gör vi en plot som visar den kurva som vi skulle få med våra startvärden, samt både den sanna och den brusiga kurvan.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y_brus, "b.", label = "med brus", markersize = 4)
ax.plot(x, y_sann, "k-", label = "original", linewidth = 1)
ax.plot(x, sinusfkt(x, *guessParams), "m-", label = "gissning", linewidth=2)
ax.legend()

Kurvanpassningen sker som tidigare med `opt.curve_fit()`. 

In [ ]:
optimParams, pcov = opt.curve_fit(f=sinusfkt, xdata=x, ydata=y_brus, p0=guessParams, method="trf", verbose=2)

Sista raden i utskriften från `opt.curve_fit()` ger oss information om anpassningen lyckades eller inte. Om inte så kan vi gå tillbaka och testa andra startvärden. Väljer vi rimliga startvärden är det mera sannolikt att vi även lyckas med kurvanpassningen.

In [ ]:
print("anpassad: A={:8g}, B={:8g}, C={:8g}".format(*optimParams))
print("original: A={:8g}, B={:8g}, C={:8g}".format(*origParams))

display(Markdown(r"$f(t) = {:8g} \sin\left( 2\pi\cdot{:+8g} t {:+8g}^o \right)$".format(*optimParams)))
display(Markdown(r"$f(t) = {:8g} \sin\left( 2\pi\cdot{:+8g} t {:+8g}^o \right)$".format(*origParams)))

Men utan att faktiskt rita upp kurvorna så är det nog inte möjligt att bedöma om python verkligen lyckades med en bra kurvanpassning.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y_brus, "b.", label = "med brus", markersize = 1)
ax.plot(x, y_sann, "k-", label = "original", linewidth=2)
ax.plot(x, sinusfkt(x, *guessParams), "m-", label = "gissning", linewidth=2)
ax.plot(x, sinusfkt(x, *optimParams), "r-", label = "anpassad", linewidth=2)
ax.text( 0.95, 0.75, 
        r"$f(t)={:.2f}\cdot\sin\left(2\pi\cdot{:.2f}t + {:.2f}^o\cdot\pi/180^o\right)$"
        .format(optimParams[0],optimParams[1],optimParams[2]), 
        horizontalalignment='right',
        size=20,
        transform=ax.transAxes )
ax.legend()